In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install -q pycaret
#!pip uninstall lightgbm -y
# install lightgbm GPU
#!pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"
# install pycaret on blazingSQL
#!/opt/conda-environments/rapids-stable/bin/python -m pip install --upgrade pycaret

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os
import sys
from requests import get

IN_COLAB = 'google.colab' in sys.modules
# kaggle環境ならTrue
if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
    env = "kaggle"
    #filename = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
    OUTPUT_DIR = './'
    data_path = '../input/feedback-prize-english-language-learning/'
# colaboratory環境ならTrue
elif IN_COLAB:
    env = "colab"
    # mount Gdrive
    from google.colab import drive
    drive.mount('/content/drive')
    #!pip install -q sentencepiece
    #!pip install -q tokenizers
    #!pip install -q transformers
    # set output directory
    #filename = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
    filename = 'stacking'
    data_path = '/content/drive/MyDrive/Kaggle/FB3/'
    OUTPUT_DIR = '/content/drive/MyDrive/Kaggle/FB3/' + filename.replace('FB3'+'-', '').replace('.ipynb', '')+'/'
else:
    env = None

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
TRAIN_DIR = data_path + 'train.csv'
TEST_DIR = data_path + 'test.csv'
SAMPLE_DIR = data_path + 'sample_submission.csv'

In [ ]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv(TRAIN_DIR)
test = pd.read_csv(TEST_DIR)
submission = pd.read_csv(SAMPLE_DIR)

print(f"train.shape: {train.shape}")
display(train.head())
print(f"test.shape: {test.shape}")
display(test.head())
print(f"submission.shape: {submission.shape}")
display(submission.head())

train.shape: (3911, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


test.shape: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


submission.shape: (3, 7)


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


## Train 2nd layer model with pycaret

### Get train data

In [ ]:
!pip install readability
!pip install autocorrect -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from autocorrect import Speller
spell = Speller(lang='en')
def misspell_counter(text):
    cnt = 0
    words = text.split(' ')
    for word in words:
        cor_word = spell(word)
        if word != cor_word:
            #print('Before: ',word,'\t' ,'After:', cor_word)
            cnt += 1
    return cnt

def unique_words_counter(text):
    cnt = 0
    words = text.split(' ')
    return len(set(words))
    

In [ ]:
import math
import re
import readability
import time
# 新たな特徴量を作成
def get_len(text):
    return len(text)

def get_readability(text):
    results = readability.getmeasures(text, lang='en')
    return results['readability grades']['ARI']

time_sta = time.time()
train['full_text_len'] = train['full_text'].map(get_len)
train['readablitiy'] = train['full_text'].map(get_readability)
train['misspell_num'] = train['full_text'].map(misspell_counter)
train['unique_num'] = train['full_text'].map(unique_words_counter)
time_end = time.time()
tim = time_end- time_sta
print('elapsed time:', tim)
display(train.head())

elapsed time: 806.3693425655365


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,full_text_len,readablitiy,misspell_num,unique_num
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,1387,30.998750,2,143
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,2635,41.487917,3,149
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,1663,79.724455,0,136
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,3973,45.320397,0,258
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,1326,36.772308,12,106


In [ ]:
train.to_pickle('/content/drive/MyDrive/Kaggle/FB3/stacking_train.pkl')

In [ ]:
PATH = '/content/drive/MyDrive/Kaggle/FB3/'
use_models = [29, 31, 2, 3, 26, 20, 21]

model_paths = [PATH+'exp'+str(i).zfill(3) for i in use_models]
oof_dfs = []
for model_path in model_paths:
    csv = pd.read_pickle(model_path+'/oof_df.pkl')
    oof_dfs.append(csv)
    print(csv.shape)


(3911, 15)
(3911, 15)
(3911, 15)
(3911, 15)
(3911, 15)
(3911, 15)
(3911, 15)


In [ ]:
len(oof_dfs)

7

In [ ]:
cols = ['pred_cohesion', 'pred_syntax', 'pred_vocabulary', 
        'pred_phraseology', 'pred_grammar', 'pred_conventions']
for idx, oof in enumerate(oof_dfs):
    for col in cols:
        train[f"pred_{col}_{idx}"] = oof[col].values

train.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,full_text_len,readablitiy,...,pred_pred_vocabulary_5,pred_pred_phraseology_5,pred_pred_grammar_5,pred_pred_conventions_5,pred_pred_cohesion_6,pred_pred_syntax_6,pred_pred_vocabulary_6,pred_pred_phraseology_6,pred_pred_grammar_6,pred_pred_conventions_6
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,1387,30.998750,...,2.980128,2.838610,2.631934,2.723644,2.733291,2.716609,2.923080,2.777478,2.597784,2.697762
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,2635,41.487917,...,2.902639,2.744143,2.555060,2.623393,2.733100,2.592851,2.916494,2.777610,2.615271,2.551598
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,1663,79.724455,...,3.283048,3.355353,3.434412,3.323067,3.262570,3.236130,3.260422,3.291111,3.311818,3.149651
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,3973,45.320397,...,3.151421,3.132882,3.244186,2.810961,2.925281,2.934289,3.153826,3.195312,3.294297,2.721133
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,1326,36.772308,...,2.457283,2.144784,1.966159,2.236139,2.316392,2.143609,2.559014,2.243440,1.959062,2.280828


In [ ]:
train.columns

Index(['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary',
       'phraseology', 'grammar', 'conventions', 'full_text_len', 'readablitiy',
       'misspell_num', 'unique_num', 'pred_pred_cohesion_0',
       'pred_pred_syntax_0', 'pred_pred_vocabulary_0',
       'pred_pred_phraseology_0', 'pred_pred_grammar_0',
       'pred_pred_conventions_0', 'pred_pred_cohesion_1', 'pred_pred_syntax_1',
       'pred_pred_vocabulary_1', 'pred_pred_phraseology_1',
       'pred_pred_grammar_1', 'pred_pred_conventions_1',
       'pred_pred_cohesion_2', 'pred_pred_syntax_2', 'pred_pred_vocabulary_2',
       'pred_pred_phraseology_2', 'pred_pred_grammar_2',
       'pred_pred_conventions_2', 'pred_pred_cohesion_3', 'pred_pred_syntax_3',
       'pred_pred_vocabulary_3', 'pred_pred_phraseology_3',
       'pred_pred_grammar_3', 'pred_pred_conventions_3',
       'pred_pred_cohesion_4', 'pred_pred_syntax_4', 'pred_pred_vocabulary_4',
       'pred_pred_phraseology_4', 'pred_pred_grammar_4',
       'pred_p

## Let's train with pycaret!
### Cohesion

In [ ]:
input_cols = [ 'cohesion', 'syntax', 'vocabulary',
       'phraseology', 'grammar', 'conventions',
       'full_text_len', 'readablitiy', 'misspell_num', 'unique_num',
       'pred_pred_cohesion_0', 'pred_pred_syntax_0', 'pred_pred_vocabulary_0',
       'pred_pred_phraseology_0', 'pred_pred_grammar_0',
       'pred_pred_conventions_0', 'pred_pred_cohesion_1', 'pred_pred_syntax_1',
       'pred_pred_vocabulary_1', 'pred_pred_phraseology_1',
       'pred_pred_grammar_1', 'pred_pred_conventions_1',
       'pred_pred_cohesion_2', 'pred_pred_syntax_2', 'pred_pred_vocabulary_2',
       'pred_pred_phraseology_2', 'pred_pred_grammar_2',
       'pred_pred_conventions_2', 'pred_pred_cohesion_3', 'pred_pred_syntax_3',
       'pred_pred_vocabulary_3', 'pred_pred_phraseology_3',
       'pred_pred_grammar_3', 'pred_pred_conventions_3']

In [ ]:
train_df = train[input_cols]
print(f"train.shape: {train_df.shape}")
display(train_df.head())

train.shape: (3911, 34)


,cohesion,syntax,vocabulary,phraseology,grammar,conventions,full_text_len,readablitiy,misspell_num,unique_num,...,pred_pred_vocabulary_2,pred_pred_phraseology_2,pred_pred_grammar_2,pred_pred_conventions_2,pred_pred_cohesion_3,pred_pred_syntax_3,pred_pred_vocabulary_3,pred_pred_phraseology_3,pred_pred_grammar_3,pred_pred_conventions_3
0,3.5,3.5,3.0,3.0,4.0,3.0,1387,30.998750,2,143,...,2.932099,2.761294,2.517373,2.755656,2.747165,2.697495,2.939530,2.735023,2.543701,2.766554
1,2.5,2.5,3.0,2.0,2.0,2.5,2635,41.487917,3,149,...,3.039436,2.782469,2.391204,2.683262,2.723522,2.664040,2.896482,2.702856,2.449788,2.708281
2,3.0,3.5,3.0,3.0,3.0,2.5,1663,79.724455,0,136,...,3.165306,3.232640,3.277324,3.106787,3.192149,3.101307,3.107142,3.231324,3.244362,2.981745
3,4.5,4.5,4.5,4.5,4.0,5.0,3973,45.320397,0,258,...,3.154569,3.211160,3.144794,2.675283,2.939273,2.856379,3.006718,3.013959,3.025609,2.516632
4,2.5,3.0,3.0,3.0,2.5,2.5,1326,36.772308,12,106,...,2.561485,2.295597,1.915890,2.519218,2.350893,2.174067,2.477248,2.201909,1.923563,2.373663


In [ ]:
from pycaret.regression import *
#set_config('seed', 2022)

In [ ]:
from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, \
    BaseShuffleSplit, _validate_shuffle_split
class MultilabelStratifiedKFold(_BaseKFold):
    """Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.
    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.
    random_state : int, RandomState instance or None, optional, default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedKFold that only uses random_state
        when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
    >>> mskf.get_n_splits(X, y)
    2
    >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
    MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
    >>> for train_index, test_index in mskf.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    Notes
    -----
    Train and test sizes may be slightly different in each fold.
    See also
    --------
    RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
    n times.
    """

    def __init__(self, n_splits=3, *, shuffle=False, random_state=None):
        super(MultilabelStratifiedKFold, self).__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def _make_test_folds(self, X, y):
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

        num_samples = y.shape[0]

        rng = check_random_state(self.random_state)
        indices = np.arange(num_samples)

        if self.shuffle:
            rng.shuffle(indices)
            y = y[indices]

        r = np.asarray([1 / self.n_splits] * self.n_splits)

        test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

        return test_folds[np.argsort(indices)]

    def _iter_test_masks(self, X=None, y=None, groups=None):
        test_folds = self._make_test_folds(X, y)
        for i in range(self.n_splits):
            yield test_folds == i

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedKFold, self).split(X, y, groups)


In [ ]:
# PyCaretは自動で入力変数の型を推定してくれる
# が、推定結果と異なる指定をする場合は、
# 引数categorical_featuresもしくは引数numeric_featuresへ明示的に指定してください。
exp_reg001 = setup(data = train_df, target = 'cohesion', session_id=2022,
                   normalize = True,
                   normalize_method = 'zscore',# {'minmax', 'robust', 'maxabs'}
                   #combine_rare_levels = True, #出現頻度がしきい値未満となるカテゴリを、全て1つのカテゴリとして統合 
                   transformation = True, 
                   ignore_low_variance = True,
                   remove_multicollinearity = True, 
                   multicollinearity_threshold = 0.98,
                   #remove_outliers = True,
                   #outliers_threshold = 0.05,
                   #create_clusters = True,# 各特徴量を用いたクラスタリングを行い、各レコードのクラスラベルを新たな特徴量として追加
                   #numeric_features=['goal_range'], # 連続値
                   #categorical_features=['category1', 'category2'], 
                   #combine_rare_levels=True, # カテゴリカルデータ
                   #rare_level_threshold=0.05,
                   #bin_numeric_features = [],
                   ignore_features = ['syntax', 'vocabulary','phraseology', 'grammar', 'conventions'], # 除外したい特徴量
                   #ordinal_features = { 'column_name' : ['low', 'medium', 'high'] }, # 順序データ
                   polynomial_features = True,
                   trigonometry_features = True,
  
                   feature_interaction = True,
                   #feature_ratio = True,
                   #pca = True, #おすすめ，特徴量の次元削減
                   #fold_strategy = 'stratifiedkfold',
                   log_experiment = True, 
                   experiment_name = 'stacking_exp001')

,Description,Value
0,session_id,2022
1,Target,cohesion
2,Original Data,"(3911, 34)"
3,Missing Values,False
4,Numeric Features,28
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(2737, 62)"


INFO:logs:Logging experiment in MLFlow
INFO:logs:SubProcess save_model() called ==================================
INFO:logs:Initializing save_model()
INFO:logs:save_model(model=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True,
                                      features_todrop=['syntax', 'vocabulary',
                                                       'phraseology', 'grammar',
                                                       'conventions'],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='cohesion',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value...
                 Fix_multicollinearity(correlation_with_targe

In [ ]:
top3 = compare_models(n_select = 3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,0.4698,0.3394,0.5816,0.1982,0.1446,0.1624,0.018
gbr,Gradient Boosting Regressor,0.4722,0.3408,0.5833,0.1929,0.1447,0.1631,3.353
br,Bayesian Ridge,0.4704,0.3423,0.5842,0.1907,0.1457,0.1622,0.023
rf,Random Forest Regressor,0.4731,0.3428,0.5848,0.1888,0.1453,0.1637,7.555
et,Extra Trees Regressor,0.4747,0.3456,0.5870,0.1827,0.1455,0.1637,2.241
ridge,Ridge Regression,0.4714,0.3458,0.5872,0.1815,0.1466,0.1621,0.018
lr,Linear Regression,0.4716,0.3459,0.5873,0.1812,0.1467,0.1621,0.360
huber,Huber Regressor,0.4723,0.3485,0.5895,0.1749,0.1470,0.1626,0.134
ada,AdaBoost Regressor,0.4799,0.3511,0.5919,0.1689,0.1478,0.1676,0.959
lightgbm,Light Gradient Boosting Machine,0.4832,0.3591,0.5986,0.1498,0.1483,0.1664,0.753


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:[OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=None,
                          normalize=True, precompute='auto', tol=None), GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=2022, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              co

In [ ]:
print(top3)

In [ ]:
models()

In [ ]:
# decision tree : top3の結果に合わせてモデルを変えてね
lasso = create_model('lasso', fold = 10)

In [ ]:
lda = create_model('lda', fold = 10)

In [ ]:
lr = create_model('lr', fold=10)

In [ ]:
tuned_lasso = tune_model(lasso, optimize='RMSE')

In [ ]:
tuned_ridge = tune_model(rd, optimize='F1')

In [ ]:
# F1 scoreで最適化されるようにハイパラチューン
tuned_lda = tune_model(lda, optimize = 'F1')

In [ ]:
# confirm hyperparameters
plot_model(tuned_rf, plot = 'parameter')

In [ ]:
ensemble_tuned_lda = ensemble_model(tuned_lda, n_estimators=50)

In [ ]:
ensemble_tumed_lr = ensemble_model(tuned_lr, n_estimators=50)

In [ ]:
ensemble_tuned_rd = ensemble_model(tuned_ridge, n_estimators=50)

In [ ]:
blender = blend_models(estimator_list = [ensemble_tumed_lr, ensemble_tuned_rd, ensemble_tuned_lda], optimize='F1', method='hard')

In [ ]:
# blend top3 models from compare_models
blender_top3 = blend_models(top3)

In [ ]:
stacker = stack_models(top3)

In [ ]:
stacker_tuned = stack_models(estimator_list = [ensemble_tumed_lr, ensemble_tuned_lda, ensemble_tuned_rd])

In [ ]:
final_model = finalize_model(blender)

In [ ]:
final_model2 = finalize_model(stacker_tuned)

In [ ]:
# デプロイ用の最終的なランダムフォレストモデルのパラメータ
print(final_model)

In [ ]:
predict_model(final_rf);

In [ ]:
save_model(final_model,OUTPUT_DIR+'/Final_Model_exp003')

## Get 2nd layer prediction

In [ ]:
# 訓練時と同じ順番に並べることを忘れずに！
np_paths = [model_path+'/preds_sigmoid.npy' for model_path in model_paths]

In [ ]:
layer1_preds = []
for path in np_paths:
    pred = np.load(path)
    layer1_preds.append(pred)

In [ ]:
def inference_pipeline(df, layer1_preds):
    df['dummy_num'] = df['html_content'].map(count_dummy)
    df['html_len'] = df['html_content'].map(get_len)
    df['goal_avg'] = df['goal'].map(get_goal_max_min)
    df['goal_range'] = df['goal'].map(get_range)

    for idx, pred in enumerate(layer1_preds):
        df[f"preds_{idx}"] = pred

    input_cols = ['dummy_num', 'goal_avg', 'goal_range', 'duration', 'html_len','category1', 'category2'] + [f"preds_{i}" for i in range(len(layer1_preds))]
    df = df[input_cols]
    return df

test = inference_pipeline(test_csv, layer1_preds)

print(f"train.shape: {test.shape}")
display(test.head())

## Final Prediction 

In [ ]:
saved_final_model = load_model(OUTPUT_DIR+'/Final_Model_exp003')

In [ ]:
unseen_predictions = predict_model(saved_final_model, data=test)
unseen_predictions.head()

## Ensemble

- average sigmoid outputs
- majority-vote label outputs
- stacking outputs

In [ ]:
from sklearn.metrics import f1_score
def get_results(oof_df, th):
    preds = oof_df['preds'].values
    targets = oof_df['state'].values
    label_preds = np.where(preds < th, 0, 1)
    score = f1_score(targets, label_preds)
    return score

In [ ]:
oof_dfs = pd.concat(oof_dfs)

In [ ]:
best_th = 0.5
best_score = 0.
for th in np.arange(0.2, 0.51, 0.0001):
    th = np.round(th, 5)
    score = get_results(oof_df, th=th)
    if best_score < score:
        best_th = th
        best_score = score
    print(f"th: {th}  score: {score:.5f}")
print(f"best_th: {best_th}  score: {best_score:.5f}")

In [ ]:
# average sigmoid outputs
avg_sig_preds = np.mean(layer1_preds, axis=0)
avg_output = np.where(avg_sig_preds<0.4842, 0, 1)

In [ ]:
# majority-vote
label_outputs_path = [path+'/submission.csv' for path in model_paths]
label_outputs = []
for path in label_outputs_path:
    csv = pd.read_csv(path, header=None)
    label_outputs.append(csv.iloc[:, 1].values)

def majority_vote(output_list):
    results = np.array(output_list)
    return [*map(lambda x: np.argmax(np.bincount(x)), results.T)]

#label_output = np.mean(label_outputs, axis=0)
#results = np.array([result_1, result_2, result_3])
label_output = majority_vote(label_outputs)

In [ ]:
# stacking outputs
stack_output = unseen_predictions['Label'].values

In [ ]:
stack_output

In [ ]:
label_output = np.array(label_output)
label_output

In [ ]:
avg_output

In [ ]:
# final prediction
final_preds = majority_vote([avg_output, label_output, stack_output])


In [ ]:
final_preds

## Submission

In [ ]:
final_preds = stack_output

In [ ]:
sample = pd.read_csv(PATH + "/sample_submit.csv",header=None)
sample.head()

In [ ]:
len(sample) == len(final_preds)

In [ ]:
sample.iloc[:, 1] = final_preds
sample.head()

In [ ]:
sample.to_csv(OUTPUT_DIR + '/submission3.csv', header=False, index=False)